# Исследования по классификации текстовых данных

## Этап 1 - Подготовка 

In [1]:
import os
from pathlib import Path    

In [2]:
DATA_DIR = Path("data")
if not DATA_DIR.exists():
    raise Exception("Data directory '%s' not found" % DATA_DIR.absolute())
    
OUT_DIR = Path("out_classification")
if not OUT_DIR.exists():
    OUT_DIR.mkdir()

In [3]:
import numpy as np
import pandas as pd

## Этап 2 - Загрузка данных

In [4]:
complaints_file = os.path.join(DATA_DIR, "complaints.xlsx")

raw_data = pd.read_excel(complaints_file, sheet_name="complaints", index_col=None)

In [5]:
raw_data.head(3)

,ISSUENUM,TOPIC,UNDER_TOPIC,REPORTER,ASSIGNEE,CREATOR,STATUS_ISSUE,CREATED,UPDATED,RESOLUTIONDATE,...,FIO_CLIENT,IIN,TEXT_CONTRACT_NUMBER,PHONE_NUMBER,INF_CLIENT_RESPONDING,INCIDENT_REGION,INCIDENT_RESOLVED,FIO_GUILT_WORK,AUTHOR_REGION,AUTHOR_SUBDIVISIONS
0,105578,ЖАЛОБА: на корреспонденцию банка,Cмс по предложениям Xsell,aoryntaeva,akalabaeva,aoryntaeva,Закрыта,2017-12-12 06:47:45,2017-12-12 12:02:50,2017-12-12 12:02:50,...,АҚЖИГИТОВ БЕЙБІТ КЕНЖЕБАЙҰЛЫ,910429301833,0,7776121291,NaN,0,Не требует создания проблемы,NaN,Алматы,Call-center
1,105582,ЖАЛОБА: на корреспонденцию банка,Cмс по предложениям Xsell,atusipova,akalabaeva,atusipova,Закрыта,2017-12-12 08:34:06,2017-12-12 15:41:16,2017-12-12 12:05:45,...,Тверитинова Анна Егоровна,761123400947,3708168706,87078452136 \t,NaN,0,Не требует создания проблемы,NaN,NaN,Call-center
2,105604,ЖАЛОБА: на обслуживание,NaN,azhalgaeva,zissayev,azhalgaeva,Закрыта,2017-12-12 09:27:28,2017-12-21 09:10:29,2017-12-14 16:05:54,...,Дарипова Айгуль Сабыргалиевна,810420403003,0,7292250739,Не дозвон,"Мангистауская область,Бейнеу, ул. Ерконай, д. 1",Не требует создания проблемы,NaN,Актау,Call-center


In [6]:
print("[i] Размерность сырых данных: ", raw_data.shape)

[i] Размерность сырых данных:  (22840, 21)


## Этап 3 - Отбор признаков

In [7]:
valuable_columns = ["ISSUENUM", "TOPIC", "DESCRIPTION"]
raw_data = raw_data[valuable_columns] # Для исследования ограничимся ID 'ISSUENUM', меткой 'TOPIC' и содержанием 'DESCRIPTION'

print("[i] : Информация о данных")
raw_data.info() # Информация о данных

[i] : Информация о данных
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22840 entries, 0 to 22839
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ISSUENUM     22840 non-null  int64 
 1   TOPIC        22805 non-null  object
 2   DESCRIPTION  22839 non-null  object
dtypes: int64(1), object(2)
memory usage: 535.4+ KB


In [8]:
raw_data.head(3)

,ISSUENUM,TOPIC,DESCRIPTION
0,105578,ЖАЛОБА: на корреспонденцию банка,Клиент просит чтобы банк не отправлял смс о сп...
1,105582,ЖАЛОБА: на корреспонденцию банка,ДД.\nПри оформлении кредита клиент предоставил...
2,105604,ЖАЛОБА: на обслуживание,Дарипова Айгуль Сабыргалиевна жалуется на мене...


## Этап 4 - Проверка на дубли и удаление строк с отсутствующими данными

In [9]:
duplicated = raw_data.loc[raw_data.duplicated(subset=['ISSUENUM'], keep=False)]
print("[i] Количество дублей идентификаторов: ", duplicated['ISSUENUM'].count())

[i] Количество дублей идентификаторов:  0


In [10]:
raw_data = raw_data.dropna(subset=['TOPIC', 'DESCRIPTION']) # Удаляем строки с пустыми значениями в столбцах 'TOPIC' и 'DESCRIPTION'

print("[i] Размерность после удаления пустых", raw_data.shape)

[i] Размерность после удаления пустых (22804, 3)


## Этап 5 - Отбор "нужных" обращений

In [11]:
#Отберем только строки, содержащие жалобы, в которых столбец 'TOPIC' содержит различные сочетания слова "жалоба"
raw_data = raw_data[(raw_data.TOPIC.str.contains("ЖАЛОБА: ") |
                     raw_data.TOPIC.str.contains("Жалоба на ") |
                     raw_data.TOPIC.str.contains("Жалобы на "))] #, na=False

print("[i] Размерность данных с Жалобами", raw_data.shape)

[i] Размерность данных с Жалобами (19415, 3)


## Этап 6 - Очистка меток от "мусора"

In [12]:
import re

In [13]:
def label_cleaner(text):
        
    text = re.sub( r'ЖАЛОБА: на ', '', text) # Убрать из темы словосочетания 'ЖАЛОБА: на ',
    text = re.sub( r'ЖАЛОБА: ', '', text) # 'ЖАЛОБА: ',
    text = re.sub( r'Жалоба на ', '', text) # 'Жалоба на ',
    text = re.sub( r'Жалобы на ', '', text) # 'Жалобы на '
    
    text = str.strip(text.lower()) #Переводим в нижний регистр и убираем пробелы в начале и конце
    return  text

In [14]:
raw_data["TOPIC"] = [label_cleaner(t) for t in raw_data["TOPIC"]] # Почистим метки от "мусора"

print("[i] Количество категорий (меток):", len(set(raw_data["TOPIC"])))
print("[i] Множество категорий (меток):")
print(set(raw_data["TOPIC"]))

[i] Количество категорий (меток): 9
[i] Множество категорий (меток):
{'не согласие с условиями договора, %%, задолженностью, штрафами, тарифами и комиссиями', 'услугу "страховку"', 'го/филиал/отделения/микроофисы/тт', 'услугу "защита семьи"', 'обслуживание', 'корреспонденцию банка', 'обслуживание в терминалах', 'услугу "хранитель"', 'карточные продукты'}


## Этап 7 - Препроцессинг

In [15]:
from nltk.stem.snowball import SnowballStemmer

In [16]:
#Пишем свой препроцессор (preprocessor)
def text_cleaner(text):
    text = text.lower() # приведение в lowercase,
    
    stemmer = SnowballStemmer("russian")
    singles = [stemmer.stem(word) for word in text.split()]
    text = ' '.join(singles)

    #Удаление незначимых слов
    stw = ['в', 'по', 'на', 'из', 'и', 'или', 'не', 'но', 'за', 'над', 'под', 'то',
           'a', 'at', 'on', 'of', 'and', 'or', 'in', 'for', 'at' ]
    remove = r'\b(' + '|'.join(stw) + r')\b'
    text = re.sub(remove,' ', text)
    
    text = re.sub( r'\b\w\b', '', text ) # удаление отдельно стоящих букв

    text = re.sub( r'\b\d+\b', ' digit ', text ) # замена цифр 

    return  text

raw_data["DESCRIPTION"] = [text_cleaner(t) for t in raw_data["DESCRIPTION"]]
raw_data["DESCRIPTION"].head(50)

0     клиент прос чтоб банк   отправля смс  спец пре...
1     дд. при оформлен кредит клиент предостав телеф...
2     дарипов айгул сабыргалиевн жал   менеджер банк...
3     добр день! клиент жалуется, что специалист гру...
4     клиент жал     что  лиц отправля смс сообщен ч...
5                                          остаток сумм
6     добр день! клиент оформ денежн кредит. менедже...
7                                              олрробро
12    при оформлен клиент был сказано, что ест акц п...
13    добр день! клиент прос   беспоко   счет спец. ...
14    дд! клиент   довол тем, чтоб получ справк необ...
15    добр день!!! клиент амирбек абза был возмущ от...
16                       кк.   предостав никак информац
17     digit    дан номер приход смс   им эдуард мур...
18    клиент жал   менеджер так как при оформлен тов...
20    клиент прос больш   присыла смс собщен спредло...
21    клиент прос больш   зван   этот номер   говар ...
22    инцидент произошел   город актоб тд мечт  

## Этап 8 - подготовка классификатора 

In [17]:
from sklearn.model_selection import KFold

In [18]:
N_FOLDS = 5
cv = KFold(n_splits=N_FOLDS, shuffle=True, random_state=0)

In [19]:
#Проверим версию scikit-learn:
import sklearn
print('[i] Версия scikit-learn: ', sklearn.__version__)

[i] Версия scikit-learn:  0.22.1


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

In [21]:
vectorizer = TfidfVectorizer() # preprocessor=text_cleaner
print(vectorizer)
print("\n")
classifier = SGDClassifier(loss='hinge', tol=0.0001) #, max_iter=20, tol=0.0001, verbose = 1
print(classifier)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.0001,
              validation_fraction=0.1, verbose=0, warm_start=False)


## Этап 9 - Обучение классификатора и тестирование

In [22]:
from sklearn.metrics import accuracy_score
print("[i] Обучение классификатора...")

text_clf = Pipeline([
                ('tfidf', vectorizer),
                ('clf', classifier),
                ])

print("[i] Тестирование...")
scores_train = []
scores_test = []

X = raw_data['DESCRIPTION'].to_list()
Y = raw_data['TOPIC'].to_list()

for train, test in cv.split(X, Y):
    X_train = [X[i] for i in train]
    Y_train = [Y[i] for i in train]
    X_test = [X[i] for i in test]
    Y_test = [Y[i] for i in test]
    
    text_clf.fit(X_train, Y_train)
    
    predicted_train = text_clf.predict(X_train)
    score = accuracy_score(Y_train, predicted_train)
    print("[i] accuracy train: ",  score)
    scores_train.append(score)
    
    predicted_test = text_clf.predict(X_test)
    score = accuracy_score(Y_test , predicted_test)
    print("[i] accuracy test: ", score)
    scores_test.append(score)
    
print("[i] средняя точность на тренировочной выборке:", np.mean(scores_train))
print("[i] средняя точность на тестовой выборке:", np.mean(scores_test))

[i] Обучение классификатора...
[i] Тестирование...
[i] accuracy train:  0.9014936904455318
[i] accuracy test:  0.8524336852948751
[i] accuracy train:  0.9026525882049962
[i] accuracy test:  0.8570692763327324
[i] accuracy train:  0.9043909348441926
[i] accuracy test:  0.8423899047128509
[i] accuracy train:  0.9019443729075457
[i] accuracy test:  0.8609322688642802
[i] accuracy train:  0.9027169714138553
[i] accuracy test:  0.8529487509657482
[i] средняя точность на тренировочной выборке: 0.9026397115632243
[i] средняя точность на тестовой выборке: 0.8531547772340973
